# Dados Visuais — ECG Image Data (Kaggle)

Instala a API do Kaggle, baixa `erhmrai/ecg-image-data`, e exibe uma imagem de exemplo para cada classe detectada.

In [1]:
# Instalações
import sys, subprocess

def pip_install(pkg: str):
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg])

for pkg in ['kaggle', 'matplotlib']:
    try:
        __import__(pkg)
    except ImportError:
        pip_install(pkg)
print('Dependências OK')

Dependências OK


In [2]:
# Autenticação Kaggle (igual ao notebook de dados numéricos)
import os, json, pathlib
home = pathlib.Path.home()
kaggle_dir = home/'.kaggle'
kaggle_dir.mkdir(exist_ok=True)
cred_path = kaggle_dir/'kaggle.json'
if os.getenv('KAGGLE_USERNAME') and os.getenv('KAGGLE_KEY') and not cred_path.exists():
    creds = {"username": os.getenv('KAGGLE_USERNAME'), "key": os.getenv('KAGGLE_KEY')}
    cred_path.write_text(json.dumps(creds))
print('Kaggle cred path:', cred_path)

Kaggle cred path: C:\Users\JuniorRodriguesdaSil\.kaggle\kaggle.json


In [ ]:
# Download temporário do dataset de imagens (com fallback offline)
import os, json, pathlib, sys, tempfile, subprocess

_tmp = tempfile.TemporaryDirectory()
vis_dir = pathlib.Path(_tmp.name) / 'visuais'
vis_dir.mkdir(parents=True, exist_ok=True)

classes_dirs = []
try:
    if os.getenv('KAGGLE_USERNAME') and os.getenv('KAGGLE_KEY'):
        kdir = pathlib.Path.home() / '.kaggle'
        kdir.mkdir(exist_ok=True)
        (kdir / 'kaggle.json').write_text(json.dumps({
            'username': os.getenv('KAGGLE_USERNAME'),
            'key': os.getenv('KAGGLE_KEY')
        }))
        cmd = [sys.executable, '-m', 'kaggle', 'datasets', 'download', '-d', 'erhmrai/ecg-image-data', '-p', str(vis_dir), '--unzip']
        print('Baixando dataset Kaggle para:', vis_dir)
        subprocess.check_call(cmd)
        # detectar subpastas como classes
        classes_dirs = [p for p in vis_dir.rglob('*') if p.is_dir() and any(i.suffix.lower() in {'.png','.jpg','.jpeg'} for i in p.iterdir() if i.is_file())]
    else:
        raise RuntimeError('Sem credenciais Kaggle')
except Exception as e:
    print('Falha ao baixar dataset Kaggle ou sem credenciais. Usando imagens sintéticas. Motivo:', e)
    # Gerar 5 classes sintéticas com imagens simples
    import numpy as np
    from PIL import Image, ImageDraw
    synth_classes = ['Normal','PVC','PAC','LBBB','RBBB']
    for cname in synth_classes:
        cdir = vis_dir / cname
        cdir.mkdir(parents=True, exist_ok=True)
        # criar 3 imagens por classe
        for k in range(3):
            img = Image.new('L', (256, 128), color=255)
            draw = ImageDraw.Draw(img)
            # Desenhar traços diferentes por classe
            for x in range(0, 256, 4):
                y = 64
                if cname == 'Normal':
                    y += int(20*np.sin(x/10))
                elif cname == 'PVC':
                    y += int(10*np.sin(x/12)) + (0 if (x//32)%8 else -30)
                elif cname == 'PAC':
                    y += int(8*np.sin(x/8)) + (0 if (x//40)%9 else -20)
                elif cname == 'LBBB':
                    y += int(15*np.sign(np.sin(x/15)))
                elif cname == 'RBBB':
                    y += int(15*np.sin(x/6))
                draw.line((x, y, x+4, y), fill=0, width=1)
            img.save(cdir / f'{cname.lower()}_{k}.png')
        classes_dirs.append(cdir)

print('Classes disponíveis:', [p.name for p in classes_dirs] or '(sintéticas geradas)')

Executando: c:\ProgramData\anaconda3\python.exe -m kaggle datasets download -d erhmrai/ecg-image-data -p data\visuais --unzip


CalledProcessError: Command '['c:\\ProgramData\\anaconda3\\python.exe', '-m', 'kaggle', 'datasets', 'download', '-d', 'erhmrai/ecg-image-data', '-p', 'data\\visuais', '--unzip']' returned non-zero exit status 1.

In [ ]:
# Exibir uma imagem de exemplo por classe
import matplotlib.pyplot as plt
import random

if not classes_dirs:
    raise RuntimeError('Nenhuma classe encontrada ou gerada.')

classes = sorted(classes_dirs, key=lambda p: p.name)
n = min(5, len(classes))
fig, axs = plt.subplots(1, n, figsize=(4*n, 4))
if n == 1:
    axs = [axs]
for ax, c in zip(axs, classes[:n]):
    imgs = [i for i in c.iterdir() if i.is_file() and i.suffix.lower() in {'.png', '.jpg', '.jpeg'}]
    if not imgs:
        ax.set_title(f'Sem imagens em {c.name}')
        ax.axis('off')
        continue
    img_path = random.choice(imgs)
    img = plt.imread(str(img_path))
    ax.imshow(img, cmap='gray')
    ax.set_title(c.name)
    ax.axis('off')
plt.tight_layout()
plt.show()